In [2]:
import tqdm
import random
import pathlib
import itertools
import collections

import os
import cv2
import numpy as np
import remotezip as rz

import tensorflow as tf

# Some modules to display an animation using imageio.
import imageio
from IPython import display
from urllib import request
from tensorflow_docs.vis import embed


In [8]:
cap = cv2.VideoCapture('/Users/kakotichi/Downloads/train_dataset_Бригады/Анализ бригад (телефон)/Есть телефон/00_51_01.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Выполните обработку каждого кадра с использованием загруженной модели
    # Например, используйте model.predict(frame) для определения объектов на кадре

    # Отображение кадра
    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [27]:
import sys

def progress_bar(count, total, status=""):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = "=" * filled_len + "-" * (bar_len - filled_len)

    sys.stdout.write("[{}] {}%  {}\r".format(bar, percents, status))
    sys.stdout.flush()


def download_progress(block_num, block_size, total_size):
    progress_bar(block_num * block_size, total_size)
    

urlretrieve(url, output_file, reporthook=download_progress)
import os
import subprocess
import tarfile
from urllib.request import urlretrieve

url = "http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar"
output_directory = "./data"
output_file = os.path.join(output_directory, "voc2012_raw.tar")
extract_directory = os.path.join(output_directory, "voc2012_raw")

# Создание папок, если они не существуют
os.makedirs(output_directory, exist_ok=True)
os.makedirs(extract_directory, exist_ok=True)

# Загрузка архива
urlretrieve(url, output_file)

# Извлечение содержимого архива
with tarfile.open(output_file, "r") as tar:
    tar.extractall(path=extract_directory)

# Вывод содержимого извлеченной папки
for item in os.listdir(os.path.join(extract_directory, "VOCdevkit/VOC2012")):
    print(item)

ImageSets====================================================] 100.0%  
SegmentationClass
SegmentationObject
Annotations
JPEGImages


In [33]:
data_root = os.path.join(root, "pew_research_data")


In [34]:
import csv

def load_file(file_path):
    data = []
    with open(file_path, encoding="utf-8") as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data.append(row)
    return data

In [35]:
file_path = os.path.join(data_root, "/Users/kakotichi/Documents/GitHub/sf_data_science/Vacation/submission.csv")
file_data = load_file(file_path)

In [ ]:
seg_dataset = VOCSegmentation(seg_root, year="2012", image_set=image_set, transforms=transformations, download=True)


In [40]:

det_dataset = VOCDetection(det_root, year="2012", image_set=image_set, transforms=transformations, download=True)


100%|██████████| 1999639040/1999639040 [22:14<00:00, 1498903.70it/s] 


Extracting ./data/voc2012_det/VOCtrainval_11-May-2012.tar to ./data/voc2012_det


In [41]:
import torch
from torch.utils.data import Dataset
from torchvision.datasets import VOCSegmentation, VOCDetection

class CombinedVOC(Dataset):
    def __init__(self, seg_root, det_root, year, image_set, transforms=None, download=True):
        self.seg_dataset = VOCSegmentation(seg_root, year=year, image_set=image_set, transforms=transforms, download=download)
        self.det_dataset = VOCDetection(det_root, year=year, image_set=image_set, transforms=transforms, download=download)
        self.transforms = transforms

    def __len__(self):
        return min(len(self.seg_dataset), len(self.det_dataset))

    def __getitem__(self, idx):
        seg_data, seg_target = self.seg_dataset[idx]
        det_data, det_target = self.det_dataset[idx]

        if self.transforms is not None:
            seg_data, seg_target = self.transforms(seg_data, seg_target)
            det_data, det_target = self.transforms(det_data, det_target)

        return (seg_data, seg_target), (det_data, det_target)

seg_root = "path/to/seg_data"
det_root = "path/to/det_data"
year = "2012"
image_set = "trainval"
transformations = None
download = True

combined_dataset = CombinedVOC(seg_root, det_root, year, image_set, transforms=transformations, download=download)


100%|██████████| 1999639040/1999639040 [23:49<00:00, 1399067.84it/s] 


Extracting path/to/seg_data/VOCtrainval_11-May-2012.tar to path/to/seg_data


100%|██████████| 1999639040/1999639040 [25:33<00:00, 1303672.42it/s] 


Extracting path/to/det_data/VOCtrainval_11-May-2012.tar to path/to/det_data


In [50]:
import cv2
import numpy as np
import urllib.request

# Загрузка модели и весов YOLOv3
yolo_config = "/Users/kakotichi/Documents/GitHub/sf_data_science/Vacation/yolov3-tiny.cfg"
yolo_weights = "/Users/kakotichi/Documents/GitHub/sf_data_science/Vacation/yolov3.weights"
net = cv2.dnn.readNet(yolo_weights, yolo_config)

# Загрузка списка классов
class_file = "/Users/kakotichi/Documents/GitHub/sf_data_science/Vacation/coco.txt"
with open(class_file, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# Получение выходных слоев для сети YOLO
layer_names = net.getLayerNames()
output_layers_indices = net.getUnconnectedOutLayers()
output_layers_indices = np.reshape(output_layers_indices, (output_layers_indices.size,))
out_layers = [layer_names[i - 1] for i in output_layers_indices]

# Устанавливаем классы, которые хотим обнаружить
classes_to_look_for = ["cell phone", "laptop", "tv", "remote"]  

In [7]:
import cv2
import numpy as np
import time

def detect_objects(frame, net, out_layers, classes_to_look_for):
    height, width, channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame, scalefactor=1/255, size=(416, 416), mean=(0, 0, 0), swapRB=True, crop=False)
    net.setInput(blob)
    layer_outputs = net.forward(out_layers)

    class_ids = []
    confidences = []
    boxes = []

    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.4 and classes[class_id] in classes_to_look_for:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                confidences.append(float(confidence))
                boxes.append([x, y, w, h])
                class_ids.append(class_id)

    indices = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.5, nms_threshold=0.4)

    detected_objects = []
    for i in indices:
        i = i[0]
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        detected_objects.append(label)
        confidence = round(confidences[i], 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, f'{label} {confidence}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
    return frame, detected_objects

yolo_config = "/Users/kakotichi/Documents/GitHub/sf_data_science/Vacation/yolov3-tiny.cfg"
yolo_weights = "/Users/kakotichi/Documents/GitHub/sf_data_science/Vacation/yolov3.weights"
net = cv2.dnn.readNet(yolo_weights, yolo_config)

class_file = "/Users/kakotichi/Documents/GitHub/sf_data_science/Vacation/coco.txt"
with open(class_file, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers_indices = net.getUnconnectedOutLayers()
output_layers_indices = np.reshape(output_layers_indices, (output_layers_indices.size,))
out_layers = [layer_names[i - 1] for i in output_layers_indices]

classes_to_look_for = ["person", "cell phone", "laptop", "tv", "remote"]

video_path = "/Users/kakotichi/Downloads/train_dataset_Бригады/Анализ бригад (телефон)/Есть телефон/02_26_36.mp4"
cap = cv2.VideoCapture(video_path)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('/Users/kakotichi/Downloads/train_dataset_Бригады/Анализ бригад (телефон)/Нет телефона/00_13_48.mp4', fourcc, 20.0, (frame_width,frame_height))

phone_in_hands = False
phone_detection_timer = None
phone_hold_time = 3.0

while cap.isOpened():
    ret, frame = cap.read()

    if ret:
        processed_frame, detected_objects = detect_objects(frame, net, out_layers, classes_to_look_for)

        if 'cell phone' in detected_objects:
            if not phone_in_hands:
                phone_detection_timer = time.time()
            phone_in_hands = True
        else:
            phone_in_hands = False

        person_in_cabin = 'person' in detected_objects

        if person_in_cabin:
            if phone_in_hands and (time.time() - phone_detection_timer) >= phone_hold_time:
                print("Телефон был в руках более 3-х секунд.")
                # Вставьте ваш код здесь, если хотите предпринять дополнительные действия после 3-х секунд
        else:
            print("Никого нет в кабине.")

        cv2.imshow('Detected Objects', processed_frame)
        out.write(processed_frame)

        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
out.release()
cv2.destroyAllWindows()

Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в кабине.
Никого нет в 